# interpretable ML 




In [ ]:
## we start again by reading our data
import pandas as pd
from sklearn.model_selection import train_test_split

eye_movements = pd.read_csv("../data/eye_movements_aggregated.csv")
eye_features = eye_movements.loc[:,['fixcount', 'firstPassCnt', 'P1stFixation', 'P2stFixation',
       'prevFixDur', 'firstfixDur', 'firstPassFixDur', 'nextFixDur',
       'firstSaccLen', 'lastSaccLen', 'prevFixPos', 'landingPos', 'leavingPos',
       'totalFixDur', 'meanFixDur', 'nRegressFrom', 'regressLen',
       'nextWordRegress', 'regressDur', 'pupilDiamMax', 'pupilDiamLag',
       'timePrtctg']]
labels = eye_movements['target'].astype(int)


In [ ]:
## multi-class with SHAP adds a layer of complexity, we will keep it simple here and 
## focus on a model separating the category 2 (relevant and correct) from the rest
labels = labels == 2
labels.value_counts()

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(eye_features, labels, 
                                                      stratify=labels, 
                                                      test_size=0.2, random_state=123)

## SHAP

In [ ]:
import shap

In [ ]:
import xgboost

model_xgb = xgboost.XGBClassifier(n_estimators=100, 
                                  max_depth=3,
                                  early_stopping_rounds=10 ,
                                  eval_metric= 'auc')
model_xgb.fit(X_train, y_train, 
              eval_set=[(X_train, y_train),(X_valid, y_valid)],
              verbose=True)



In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score( y_valid , model_xgb.predict_proba(X_valid)[:,1] )

In [ ]:
eye_features.head()

## SHAP basic usage and plots

In [ ]:
import shap

%time explainer = shap.Explainer( model_xgb ) ## creates an explainer from our model
%time shap_values = explainer(X_train) ## compute shap values for the prediction of the models on some data

In [ ]:
explainer

In [ ]:
shap_values

In [ ]:
shap_values[:,'fixcount']

In [ ]:
shap_values[:,'fixcount'].values.shape

In [ ]:
y_train[:5]

In [ ]:
model_xgb.predict( X_train.iloc[:,:] )

In [ ]:
## explaination of a single prediction

## try with 0 , 1 ,19 ,25 ,26
shap.plots.waterfall(shap_values[1])

In [ ]:
## average SHAP value over all predictions --> proxy of feature importance 
shap.plots.bar(shap_values)

In [ ]:
## beeswarm gives us a sense of the relationship between the feature values and the SHAP
shap.plots.beeswarm(shap_values)

In [ ]:
## plotting the SHAP value for each observed age
shap.plots.scatter( shap_values[:,'P2stFixation'] )

In [ ]:
shap.plots.scatter( shap_values[:,'pupilDiamMax'] )

In [ ]:
shap.plots.scatter( shap_values[:,'timePrtctg'] )

In [ ]:
## visualize some of the interaction between feature by coloring according to another variable 
shap.plots.scatter( shap_values[:,'prevFixPos'] , color=shap_values )

In [ ]:
## If you give all shap_values, the library searched for the one which may interact the most
shap.plots.scatter( shap_values[:,'prevFixPos'] , color=shap_values[:,'totalFixDur'] )

## SHAP interaction values

In [ ]:
%%time
shap_interaction_values = explainer.shap_interaction_values(X_train)

In [ ]:
import seaborn as sns
sns.heatmap( shap_interaction_values[1], 
            xticklabels = X_train.columns,
            yticklabels = X_train.columns,
           center = 0, cmap = 'bwr')

In [ ]:
shap_values[1].values

In [ ]:
shap_interaction_values[1].sum(axis = 1)

In [ ]:
clustering = shap.utils.hclust(X_train, y_train)
shap.plots.bar(shap_values, clustering=clustering,clustering_cutoff=1.8)

### exercise

We train a similar model, but with max_depth fixed to 1.

Look at the corresponding SHAP values. What do you see?


In [ ]:
model_xgb2 = xgboost.XGBClassifier(n_estimators=100, 
                                  max_depth=1,
                                  early_stopping_rounds=10 ,
                                  eval_metric= 'auc')
model_xgb2.fit(X_train, y_train, 
              eval_set=[(X_train, y_train),(X_valid, y_valid)],
              verbose=True)


In [ ]:
explainer2 = shap.Explainer( model_xgb2 ) ## creates an explainer from our model
shap_values2 = explainer2(X_train) ## compute shap values for the prediction of the models on some data

In [ ]:
shap.plots.beeswarm(shap_values2)

In [ ]:
shap.plots.scatter( shap_values2[: , "P2stFixation"] )

In [ ]:
max_depth = 1 # -> tree has single node

### different explainer types

In [ ]:
%%time
explainer = shap.Explainer( model_xgb2 ) ## defaults to TreeExplainer
print(type(explainer))
# TreeExplainer: 
#  * explores the tree ensemble to compute the SHAP values efficiently
#  * somewhat built-in the trees libraries nowadays -> extra fast computation
shap_values = explainer(X_train) 


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


Xt_scaled = StandardScaler().fit_transform(X_train)

LR = LogisticRegression()
LR.fit(Xt_scaled, y_train)


In [ ]:
%%time

explainer = shap.Explainer( LR ) 
shap_values = explainer(Xt_scaled) 

explainer

In [ ]:
%%time
explainer = shap.Explainer( LR.predict_proba , Xt_scaled[:100,:] ) 
shap_values = explainer(Xt_scaled[:100,:]) 

explainer

[permutation explainer](https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/explainers/Permutation.html)
 * uses smart permutation scheme to estimate SHAP values
 * needs a "background" sample which is used to "simulate" values for the masked features

An alternative is to explicitely say that it is a Linear model:

In [ ]:
%%time
# explain the model's predictions using SHAP
explainer = shap.explainers.Linear(LR, Xt_scaled[:100,:])
shap_values = explainer(Xt_scaled[:100,:])

In [ ]:
explainer

But in some case you won't have a choice and you need to go for a "generic" explainer such as PermutationExplainer

For example, for a KNN:

In [ ]:
%%time 
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=10)
KNN.fit(Xt_scaled, y_train)


explainer = shap.Explainer( KNN.predict_proba , Xt_scaled[:100,:]) 
shap_values = explainer(Xt_scaled[:100,:]) 

See the API documentation for [other "explainers"](https://shap.readthedocs.io/en/latest/api.html#explainers), either generic or adapted to specific kind of models.


### causation, correlation, and interpretation of predictive models

We are sure you have heard it a large number of time, but this message really is really worth repeating.

The SHAP documentation actually proposes an [insightful article](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/Be%20careful%20when%20interpreting%20predictive%20models%20in%20search%20of%20causal%20insights.html) on the topic of misleading interpretation.

Our advise is to read it attentively, meditate on the subject, and then read it again.



## LIME

In [ ]:
#!pip install lime

In [ ]:
import numpy as np
from lime import lime_tabular


In [ ]:
%%time
explainer = lime_tabular.LimeTabularExplainer( np.array( X_train ) , 
                                                   feature_names=X_train.columns, 
                                                   class_names=['Incorrect','Correct'], 
                                                   discretize_continuous=True)

In [ ]:
isCorrect = model_xgb.predict( X_valid ) == y_valid

print("correct positive cases:")
print( np.where( isCorrect * (y_valid==1) )[0][:5] )

print("correct negative cases:")
print( np.where( isCorrect * (y_valid==0) )[0][:5] )


print("incorrect positive cases:")
print( np.where( (~isCorrect) * (y_valid==1) )[0][:5] )

print("incorrect negative cases:")
print( np.where( (~isCorrect) * (y_valid==0) )[0][:5] )




In [ ]:
i = 1
exp = explainer.explain_instance(np.array( X_valid.iloc[i,:] ) , 
                                 model_xgb.predict_proba, 
                                 num_features=10, top_labels=1)
exp.show_in_notebook(show_table=True, show_all=False)

In [ ]:
exp

In [ ]:
exp.as_list(0)